In [ ]:
# Install required packages
!pip install pyspark nltk

In [ ]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import (regexp_replace, col, lower, current_timestamp, when, count, size, window, round, udf)
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.types import DoubleType
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import os
import glob
from time import sleep
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display, clear_output

In [ ]:
# Initialize Spark session
spark = SparkSession.builder.appName("SentimentAnalysis").getOrCreate()

# Download VADER lexicon
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
# Function to rename columns
def rename_columns(df):
    """
    Renames specific columns in the dataframe for consistency and clarity.

    Args:
        df (DataFrame): Input Spark DataFrame with raw column names.

    Returns:
        DataFrame: Spark DataFrame with renamed columns.
    """
    return df.withColumnRenamed("message to examine", "text")\
             .withColumnRenamed("label (depression result)", "label")

# Function to clean text
def clean_text(df):
    """
    Performs text cleaning operations such as converting to lowercase,
    removing URLs, mentions, hashtags, special characters, and extra spaces.

    Args:
        df (DataFrame): Input Spark DataFrame with text data.

    Returns:
        DataFrame: Spark DataFrame with a cleaned text column.
    """
    return (
        df.withColumn("cleaned_text", lower(col("text")))
          .withColumn("cleaned_text", regexp_replace(col("cleaned_text"), r"http\S+", ""))  # Remove URLs
          .withColumn("cleaned_text", regexp_replace(col("cleaned_text"), r"@\w+", ""))     # Remove mentions
          .withColumn("cleaned_text", regexp_replace(col("cleaned_text"), r"#\w+", ""))     # Remove hashtags
          .withColumn("cleaned_text", regexp_replace(col("cleaned_text"), r"[^a-zA-Z\s]", ""))  # Remove special characters
          .withColumn("cleaned_text", regexp_replace(col("cleaned_text"), r"\s+", " "))     # Remove extra spaces
    )

# Function to preprocess text: Tokenization and stop word removal
def preprocess_text(df):
    """
    Tokenizes the cleaned text and removes stop words. Filters out rows
    with empty token lists.

    Args:
        df (DataFrame): Input Spark DataFrame with cleaned text.

    Returns:
        DataFrame: Spark DataFrame with tokens and filtered tokens.
    """
    tokenizer = Tokenizer(inputCol="cleaned_text", outputCol="tokens")
    stop_words_remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens")

    # Apply transformations
    df_tokenized = tokenizer.transform(df)
    df_filtered = stop_words_remover.transform(df_tokenized)

    # Remove rows with empty tokens
    return df_filtered.filter(size(col("filtered_tokens")) > 0)

# Define the VADER sentiment analysis UDF
def analyze_sentiment_vader(text):
    """
    Performs sentiment analysis using the VADER lexicon. Calculates a compound
    score and classifies the sentiment as positive (1.0) or negative (0.0).

    Args:
        text (str or list): Input text or tokenized list of words.

    Returns:
        float: Sentiment classification, where 1.0 is positive and 0.0 is negative.
    """
    if not text:
        return 0  # if text is empty
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(' '.join(text) if isinstance(text, list) else text)
    compound_score = sentiment_scores['compound']
    return 1.0 if compound_score < -0.25 else 0.0 # Threshold is set to -0.25

# Register UDF for Spark
vader_sentiment_udf = udf(analyze_sentiment_vader, DoubleType())

# Load and preprocess data
def process_batch_data(file_path):
    """
    Loads the data from a CSV file, renames columns, and performs cleaning
    and preprocessing.

    Args:
        file_path (str): Path to the input CSV file.

    Returns:
        DataFrame: Processed Spark DataFrame ready for sentiment analysis.
    """
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    df = rename_columns(df)
    df = clean_text(df)
    df = preprocess_text(df)
    return df

# Sentiment analysis and evaluation
def apply_vader_sentiment(df):
    """
    Applies the VADER sentiment analysis UDF to classify text sentiment.

    Args:
        df (DataFrame): Spark DataFrame with filtered tokens.

    Returns:
        DataFrame: Spark DataFrame with an additional column for predicted sentiment.
    """
    return df.withColumn("vader_predicted_label", vader_sentiment_udf(col("filtered_tokens")))

In [ ]:
# Main analysis process
file_path = "/content/sentiment_tweets3(1).csv"
df_processed = process_batch_data(file_path)
df_with_vader = apply_vader_sentiment(df_processed)

# Calculate accuracy
accuracy = (
    df_with_vader.filter(df_with_vader.vader_predicted_label == df_with_vader.label).count() /
    df_with_vader.count()
)
print(f"VADER Accuracy: {accuracy * 100:.2f}%")

df_eval = df_with_vader.select("label", "vader_predicted_label").toPandas()
df_eval["vader_predicted_label"] = df_eval["vader_predicted_label"].round().astype(int)
df_eval = df_eval[df_eval["label"].apply(lambda x: str(x).isdigit())]
df_eval = df_eval[df_eval["vader_predicted_label"].apply(lambda x: str(x).replace(".0", "").isdigit())]
df_eval["label"] = df_eval["label"].astype(int)

# Step 3: Calculate recall and F1-score
from sklearn.metrics import classification_report, recall_score, f1_score

print(classification_report(df_eval["label"], df_eval["vader_predicted_label"], digits=4))

recall = recall_score(df_eval["label"], df_eval["vader_predicted_label"])
f1 = f1_score(df_eval["label"], df_eval["vader_predicted_label"])
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

VADER Accuracy: 87.64%
              precision    recall  f1-score   support

           0     0.9110    0.9329    0.9218      7973
           1     0.7468    0.6846    0.7144      2305

    accuracy                         0.8772     10278
   macro avg     0.8289    0.8087    0.8181     10278
weighted avg     0.8741    0.8772    0.8753     10278

Recall: 0.6846
F1 Score: 0.7144


In [ ]:
# Real-time plotting function
def plot_sentiment_analysis(aggregated_data):
    """
    Generates and updates a real-time Plotly chart to display sentiment analysis results.

    Args:
        aggregated_data (list): List of dictionaries containing sentiment percentages
                                and timestamps for visualization.
    """
    plot_times, positive_percentages, negative_percentages = [], [], []
    fig = make_subplots(rows=1, cols=1)
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines+markers', name='Positive Sentiment (%)', line=dict(color='green')))
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines+markers', name='Negative Sentiment (%)', line=dict(color='red')))
    fig.update_layout(title='Sentiment Analysis Over Time', xaxis_title='Time', yaxis_title='Sentiment Percentage', template='plotly_white')

    for entry in aggregated_data:
        plot_times.append(entry['time'])
        positive_percentages.append(entry['positive_percentage'])
        negative_percentages.append(entry['negative_percentage'])

        fig.data[0].x = plot_times
        fig.data[0].y = positive_percentages
        fig.data[1].x = plot_times
        fig.data[1].y = negative_percentages

        clear_output(wait=True)
        display(fig)

In [ ]:
# Simulated streaming processing (5 minutes interval)
def simulated_streaming_processing(data, chunk_size, output_path, window_size="15 seconds", toPlot=True, toAggregate=False):
    """
    Processes data in a simulated streaming fashion. Aggregates sentiment analysis
    results in time windows and optionally generates real-time plots.

    Args:
        data (DataFrame): Pandas DataFrame containing input data.
        chunk_size (int): Number of rows to process per chunk.
        output_path (str): Path to store aggregated results as a CSV file.
        window_size (str): Time window size for aggregation.
        toPlot (bool): If True, plots results in real-time.
        toAggregate (bool): If True, displays aggregated results.
    """
    start_time = pd.to_datetime("2024-12-02 22:00:00")
    end_time = start_time + pd.Timedelta(seconds=15)
    aggregated_results = []

    for i in range(0, len(data), chunk_size):
        # Extract chunk
        chunk = data.iloc[i:i + chunk_size]
        chunk['timestamp'] = start_time
        start_time = end_time
        end_time = start_time + pd.Timedelta(seconds=15)

        # Process data chunk
        chunk_spark = spark.createDataFrame(chunk)
        chunk_spark = rename_columns(chunk_spark)
        chunk_spark = clean_text(chunk_spark)
        chunk_spark = preprocess_text(chunk_spark)

        # Apply sentiment analysis
        df_with_vader = apply_vader_sentiment(chunk_spark)
        df_with_vader = df_with_vader.withColumn("processing_time", current_timestamp())

        # Aggregate in 5-minute windows
        df_aggregated = (
            df_with_vader
            .groupBy(window(col("processing_time"), window_size))
            .agg(
                count(when(col("vader_predicted_label") == 1.0, True)).alias("positive_count"),
                count(when(col("vader_predicted_label") == 0.0, True)).alias("negative_count"),
                count("*").alias("total_count")
            )
        )

        # Calculate percentages
        df_aggregated_with_percentages = df_aggregated.withColumn(
            "positive_percentage", round((col("positive_count") / col("total_count")) * 100, 1)
        ).withColumn(
            "negative_percentage", round((col("negative_count") / col("total_count")) * 100, 1)
        )

        df_aggregated_with_times = df_aggregated_with_percentages.select(
            "window.start", "window.end", "positive_count", "negative_count",
            "total_count", "positive_percentage", "negative_percentage"
        )

        # Convert to Pandas DataFrame for storage or visualization
        pandas_df = df_aggregated_with_times.toPandas()

        if os.path.exists(output_path):
            pandas_df.to_csv(output_path, mode='a', header=False, index=False)
        else:
            pandas_df.to_csv(output_path, mode='w', header=True, index=False)

        # Append results for plotting
        for _, row in pandas_df.iterrows():
            aggregated_results.append({
                'time': row['start'],
                'positive_percentage': row['positive_percentage'],
                'negative_percentage': row['negative_percentage']
            })

        # Whether to plot or aggregate (one at a time, since we're using cloud-based notebook)
        if toAggregate:
          df_aggregated_with_percentages.show(truncate=False)
        elif toPlot:
          plot_sentiment_analysis(aggregated_results)

        # Delay of 5 minutes to simulate real-time
        sleep(5)

def clean_output_folder(output_folder):
    """
    Cleans the output folder by removing all files.

    Args:
        output_folder (str): Path to the folder to be cleaned.
    """
    files = glob.glob(f"{output_folder}/*")
    for file in files:
        os.remove(file)
        print(f"Deleted {file}")

In [ ]:
clean_output_folder("output")
data = pd.read_csv(file_path)
output_path = "/content/output"
# Aggregate the streaming data results
simulated_streaming_processing(data, chunk_size=100, output_path=output_path, toPlot=False, toAggregate=True)

<ipython-input-15-31d0ec190a46>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['timestamp'] = start_time


+------------------------------------------+--------------+--------------+-----------+-------------------+-------------------+
|window                                    |positive_count|negative_count|total_count|positive_percentage|negative_percentage|
+------------------------------------------+--------------+--------------+-----------+-------------------+-------------------+
|{2025-06-04 16:20:30, 2025-06-04 16:20:45}|6             |93            |99         |6.1                |93.9               |
+------------------------------------------+--------------+--------------+-----------+-------------------+-------------------+



<ipython-input-15-31d0ec190a46>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['timestamp'] = start_time


+------------------------------------------+--------------+--------------+-----------+-------------------+-------------------+
|window                                    |positive_count|negative_count|total_count|positive_percentage|negative_percentage|
+------------------------------------------+--------------+--------------+-----------+-------------------+-------------------+
|{2025-06-04 16:20:45, 2025-06-04 16:21:00}|6             |93            |99         |6.1                |93.9               |
+------------------------------------------+--------------+--------------+-----------+-------------------+-------------------+



<ipython-input-15-31d0ec190a46>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['timestamp'] = start_time


+------------------------------------------+--------------+--------------+-----------+-------------------+-------------------+
|window                                    |positive_count|negative_count|total_count|positive_percentage|negative_percentage|
+------------------------------------------+--------------+--------------+-----------+-------------------+-------------------+
|{2025-06-04 16:21:00, 2025-06-04 16:21:15}|5             |94            |99         |5.1                |94.9               |
+------------------------------------------+--------------+--------------+-----------+-------------------+-------------------+



<ipython-input-15-31d0ec190a46>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['timestamp'] = start_time
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
clean_output_folder("output")
data = pd.read_csv(file_path)
output_path = "/content/output"
# Plot the streaming data results
simulated_streaming_processing(data, chunk_size=100, output_path=output_path, toPlot=True, toAggregate=False)

<ipython-input-15-31d0ec190a46>:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# Stop Spark session
spark.stop()